In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [72]:
df = pd.read_csv('Cardiovascular_Disease_Dataset.csv')
df.head()

,patientid,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
0,103368,53,1,2,171,0,0,1,147,0,5.3,3,3,1
1,119250,40,1,0,94,229,0,1,115,0,3.7,1,1,0
2,119372,49,1,2,133,142,0,0,202,1,5.0,1,0,0
3,132514,43,1,0,138,295,1,1,153,0,3.2,2,2,1
4,146211,31,1,1,199,0,0,2,136,0,5.3,3,2,1


In [79]:
# patient id is not relevant
# df.drop('patientid', axis=1, inplace=True)
X = df.drop(['target'], axis=1).values
y = df['target'].values.reshape(-1, 1)

In [80]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [81]:
# nn class
class NeuralNetwork:
    def __init__(self, layer_sizes):
        self.weights = [np.random.randn(size, next_size) * np.sqrt(2. / size) for size, next_size in zip(layer_sizes[:-1], layer_sizes[1:])]
        self.biases = [np.zeros((1, next_size)) for next_size in layer_sizes[1:]]

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def feedforward(self, a):
        print("Input shape:", a.shape)  # Diagnostic print
        activations = [a]
        zs = []
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            print(f"Layer {i}: weights shape {w.shape}, biases shape {b.shape}")  # Diagnostic print
            z = np.dot(a, w) + b
            zs.append(z)
            a = self.sigmoid(z)
            activations.append(a)
        return activations, zs

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activations, zs = self.feedforward(x)
        delta = (activations[-1] - y) * self.sigmoid_derivative(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(activations[-2].T, delta)
        for l in range(2, len(self.weights) + 1):
            z = zs[-l]
            sp = self.sigmoid_derivative(z)
            delta = np.dot(delta, self.weights[-l+1].T) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(activations[-l-1].T, delta)
        return nabla_b, nabla_w

    def train(self, X, y, epochs, lr):
        for epoch in range(epochs):
            nabla_b = [np.zeros(b.shape) for b in self.biases]
            nabla_w = [np.zeros(w.shape) for w in self.weights]
            delta_nabla_b, delta_nabla_w = self.backprop(X, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            self.weights = [w - (lr / len(X)) * nw for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b - (lr / len(X)) * nb for b, nb in zip(self.biases, nabla_b)]
            if epoch % 100 == 0:
                loss = np.mean((self.feedforward(X)[0][-1] - y) ** 2)
                print(f"Epoch {epoch}, Loss: {loss}")

In [84]:
# train the neural network
nn = NeuralNetwork([X_train.shape[1], 64, 1])
nn.train(X_train, y_train, epochs=1000, lr=0.01)

Input shape: (800, 13)
Layer 0: weights shape (13, 64), biases shape (1, 64)
Layer 1: weights shape (64, 1), biases shape (1, 1)
Input shape: (800, 13)
Layer 0: weights shape (13, 64), biases shape (800, 64)
Layer 1: weights shape (64, 1), biases shape (800, 1)
Epoch 0, Loss: 0.30424034659659205
Input shape: (800, 13)
Layer 0: weights shape (13, 64), biases shape (800, 64)
Layer 1: weights shape (64, 1), biases shape (800, 1)
Input shape: (800, 13)
Layer 0: weights shape (13, 64), biases shape (800, 64)
Layer 1: weights shape (64, 1), biases shape (800, 1)
Input shape: (800, 13)
Layer 0: weights shape (13, 64), biases shape (800, 64)
Layer 1: weights shape (64, 1), biases shape (800, 1)
Input shape: (800, 13)
Layer 0: weights shape (13, 64), biases shape (800, 64)
Layer 1: weights shape (64, 1), biases shape (800, 1)
Input shape: (800, 13)
Layer 0: weights shape (13, 64), biases shape (800, 64)
Layer 1: weights shape (64, 1), biases shape (800, 1)
Input shape: (800, 13)
Layer 0: weight

In [85]:
print("X_test shape:", X_test.shape)
y_pred = nn.feedforward(X_test)[0][-1]
y_pred = [1 if i > 0.5 else 0 for i in y_pred]
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

X_test shape: (200, 13)
Input shape: (200, 13)
Layer 0: weights shape (13, 64), biases shape (800, 64)


ValueError: operands could not be broadcast together with shapes (200,64) (800,64) 